In [1]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


In [2]:
seed = 0

import random
import numpy as np
from tensorflow import set_random_seed

random.seed(seed)
np.random.seed(seed)
set_random_seed(seed)

In [3]:
train = pd.read_csv('FrenchData.csv')
train.drop(columns=['Unnamed: 0'], axis=1,inplace=True)
test = pd.read_csv('test.csv')
pd.set_option('display.max_colwidth',-1)
test=test[['article']]

In [7]:
train.head()

,tweet,polarity
1163,"Oh, attendez qu'il ait 1 en Oakland le 18. Ne comprend pas pourquoi il n'a que 1 & amp; En chêne\n",0
1164,Est absent de parler à mon bff sur le téléphone !!!!\n,0
1165,"Je me demande pourquoi je suis si stupide besoin de manger plus de poisson, je suppose * soupir * ...\n",0
1166,Je manque à mes anciens amis de l'école primaire et secondaire\n,0
1167,Corrigez ceci parce qu'il freine toutes les recherches externes de l'avatar Twitter\n,0


In [28]:
for ind in train.index:
    if(train.at[ind,'polarity']!='0')and(train.at[ind,'polarity']!='4'):
        train=train.drop([ind],axis="index")

        
train['polarity'].value_counts()

4    132483
0    122196
Name: polarity, dtype: int64

In [30]:
for ind in train.index:
    if(train.at[ind,'polarity']=='4'):
        train.at[ind,'polarity']=1
    else:
        train.at[ind,'polarity']=0

In [31]:
train['polarity'].value_counts()

1    132483
0    122196
Name: polarity, dtype: int64

In [32]:
def format_data(train, test, max_features, maxlen):
    """
    Convert data to proper format.
    1) Shuffle
    2) Lowercase
    3) Sentiments to Categorical
    4) Tokenize and Fit
    5) Convert to sequence (format accepted by the network)
    6) Pad
    7) Voila!
    """
    from keras.preprocessing.text import Tokenizer
    from keras.preprocessing.sequence import pad_sequences
    from keras.utils import to_categorical
    
    train = train.sample(frac=1).reset_index(drop=True)
    train['tweet'] = train['tweet'].apply(lambda x: x.lower())
    test['article'] = test['article'].apply(lambda x: x.lower())

    X = train['tweet']
    test_X = test['article']
    Y = to_categorical(train['polarity'].values)

    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(X))

    X = tokenizer.texts_to_sequences(X)
    X = pad_sequences(X, maxlen=maxlen)
    test_X = tokenizer.texts_to_sequences(test_X)
    test_X = pad_sequences(test_X, maxlen=maxlen)

    return X, Y, test_X

In [33]:
maxlen = 125
max_features = 10000

X, Y, test_X = format_data(train, test, max_features, maxlen)

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.25, random_state=seed)

In [35]:
model = Sequential()

# Input / Embdedding
model.add(Embedding(max_features,100,mask_zero=True))
model.add(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model.add(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))

# Output layer
model.add(Dense(2, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
lstm_5 (LSTM)                (None, None, 64)          42240     
_________________________________________________________________
lstm_6 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 1,054,722
Trainable params: 1,054,722
Non-trainable params: 0
_________________________________________________________________


In [36]:
epochs = 1
batch_size = 32


In [37]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=epochs, batch_size=batch_size, verbose=1)

Train on 191009 samples, validate on 63670 samples
Epoch 1/1
191009/191009 [==============================] - 1645s 9ms/step - loss: 0.4836 - acc: 0.7686 - val_loss: 0.4455 - val_acc: 0.7917


In [38]:
test['Sentiment'] = model.predict_classes(test_X, batch_size=batch_size, verbose=1)

346/346 [==============================] - 2s 5ms/step


In [39]:
test


,article,Sentiment
0,"l’assemblée générale des nations-unies, a organisé une session extraordinaire, le jeudi 1er août 2019, à new york, qui a été consacrée au défunt président tunisien, ,mohamed béji caïd essebsi,, et à laquelle ont assisté le secrétaire général des nations-unies, ,antonio guterres,, et la présidente de l’assemblée générale, ,maria fernanda espinosa,.,au début de cette séance, les représentants des états membres de l’organisation des nations-unies, ont observé une minute de silence en hommage à la mémoire du défunt président de la république.,l’ambassadeur chargé d’affaires par intérim de la mission de la république tunisienne auprès de l’organisation des nations unies, a prononcé l’allocution du ministre des affaires étrangères, ,khémaïes jhinaoui,, lequel n’avait pu se déplacer à new york et se joindre à cette cérémonie, en raison de l’engagement des membres du gouvernement à prendre part à la période de deuil national, ainsi que sa présidence de la réunion exceptionnelle entrant dans le cadre de 37ème édition de la conférence annuelle des chefs des missions diplomatiques, permanentes et consulaires.,en prenant la parole, antonio guterres a rendu un vibrant hommage à la mémoire de béji caïd essebsi, qu’il a qualifié de « ,grand homme ,d’état, » tout en soulignant qu’il s’agit d’une grande perte pour la tunisie comme pour toute la région.,il a ajouté « ,alors que la tunisie pleure la disparition de ce pionnier, les tunisiens peuvent être fiers du riche héritage qu’il laisse derrière lui., ,a noter que le drapeau de l’onu a été mis en berne, durant la journée du 1er août, dans toutes les représentations des nations-unies dans le monde.",1
1,"une ,cérémonie d’investiture, du président de la république provisoire, ,mohamed ennaceur,, a été organisée au palais de carthage, dans la matinée du 1er août 2019, lors de laquelle le chef de l’état, a salué le drapeau national, sur les airs rythmés de l’hymne national, ainsi qu’une composition des trois corps d’armée., , , ,le président de la république saluant mm. habib essid et lazhar karoui chebbi ,",1
2,"\r\r\n\tquelques heures après avoir assisté aux funérailles nationales du président beji caid essebsi pendant lesquelles il a fait l’éloge du défunt, le président français emmanuel macron a posté un tweet dans lequel il a évoqué toute son émotion.,\r\r\n\tdans son tweet, macron évoque également toute son admiration face au peuple tunisien et à la nouvelle tunisie fière et démocratique.,\r\r\n\t ,\r\r\n\t\tla tunisie perd un grand président. nous perdons un ami. les combats de béji caïd essebsi ne sont pas terminés. c’est au peuple tunisien qu’il revient aujourd’hui de continuer à porter une tunisie ouverte, tolérante et patriotique. une tunisie qui aime la liberté et la défend.",1
3,"\r\r\n\tle ministre de la défense abdelkrim zbidi est revenu sur la dernière rencontre avec le président défunt beji caid essesbi au palais de carthage lundi dernier 22 juillet.,\r\r\n\tle ministre a assuré que le défunt avait essayé de cacher avec beaucoup de tact son état de santé mais lui avait finalement déclaré « je sais que mon heure est arrivée et nous sommes préparés à la volonté de dieu ».,\r\r\n\tzbidi a ajouté que beji lui avait présenté beaucoup de messages positifs mais avait considéré que le temps n’est pas adéquat à les faire connaitre médiatiquement, selon lui.,\r\r\n\t",0
4,"\r\r\n\tle président syrien a envoyé un télégramme de condoléances aux autorités tunisiennes pour le décès du président beji caid essebsi.,\r\r\n\tla correspondance parvient six jours après le décès du défunt beji et malgré une rupture diplomatique entre tunis et damas depuis 2012 qui a entraîné l’absence de la syrie au sommet arabe organisé à tunis malgré les efforts déployés pour la réconciliation.,\r\r\n\tdans ce message, le président syrien a déclaré « j’ai appris avec une immense tristesse le décès du président de la république tunisienne mohamed beji caid essebsi qui a travaillé toute sa vie p